In [28]:
import pandas as pd
import numpy as np
from pymoo.algorithms.unsga3 import UNSGA3
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination

from notebooks.optimization_problems.constraints import Requirements

In [29]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/eo_telesat_post_processed.h5"

row_limit = -1

store = pd.HDFStore(file_path)

instances = store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances = instances.sort_index(0)

In [30]:
case, target = pickle.load(open('raw/eo_settings.pkl', 'rb'))

In [31]:
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.fc_Hz = target['frequency']
sys_param.GT_dBK = target['GT_dBK']
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('../dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df[['isend', 'isend_200MHz', 'isend_300MHz']].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = case['requirements']
#requirements.max_energy = -1

In [32]:
from combined_problem import ExtendedCombinedProblem
problem = ExtendedCombinedProblem(instances, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3]))

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 100, 'n_offsprings': 25, 'n_max_gen': 5000}

settings = {}

# U-NSGA-III
from pymoo.factory import get_reference_directions, get_visualization, get_termination

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=14)
print("# dirs: %d" % len(ref_dirs))

settings['UNSGA-III'] = {
    'label': 'unsga3',
    'algorithm': UNSGA3(
        pop_size=int(np.ceil(len(ref_dirs) / 100) * 100),
        n_offsprings=int(np.ceil(len(ref_dirs) / 100) * 100 / 4),
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    # 'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
    #                                                   n_last=30,
    #                                                   nth_gen=5,
    #                                                   n_max_gen=algo_settings['n_max_gen'],
    #                                                   n_max_evals=None)
    'termination': get_termination("time", "01:00:00")
}

setting = settings['UNSGA-III']

# dirs: 120


In [33]:
termination = setting['termination']
algorithm = setting['algorithm']

n_gen_inc = 4950

In [34]:
from os import path

file_name = 'raw/eo_Telesat_%s_%d_%d.pkl' % (setting['label'], algorithm.pop_size, algorithm.n_offsprings)

if path.exists(file_name):
    problem, setting, res = pickle.load(open(file_name, 'rb'))
    algorithm = res.algorithm
    algorithm.has_terminated = False
    n_gen = res.algorithm.n_gen
else:
    n_gen = 0

In [35]:
n_gen = n_gen + n_gen_inc
print(n_gen)
termination = ('n_gen', n_gen)

res = minimize(problem,
               algorithm,
               termination,
               #termination=termination,
               seed=1,
               #save_history=True,
               verbose=True,
               copy_algorithm=False,
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

pickle.dump((problem, setting, res), open(file_name, 'wb'))

5000
   51 |    2700 |  0.00000E+00 |  0.00000E+00 |      17 |  0.067846453 |        nadir
   52 |    2750 |  0.00000E+00 |  0.00000E+00 |      17 |  0.063535776 |        nadir
   53 |    2800 |  0.00000E+00 |  0.00000E+00 |      21 |  0.051589693 |            f
   54 |    2850 |  0.00000E+00 |  0.00000E+00 |      18 |  0.014066525 |            f
   55 |    2900 |  0.00000E+00 |  0.00000E+00 |      20 |  0.035378749 |            f
   56 |    2950 |  0.00000E+00 |  0.00000E+00 |      18 |  0.022917809 |            f
   57 |    3000 |  0.00000E+00 |  0.00000E+00 |      23 |  0.008505543 |        ideal
   58 |    3050 |  0.00000E+00 |  0.00000E+00 |      22 |  0.007472712 |        ideal
   59 |    3100 |  0.00000E+00 |  0.00000E+00 |      24 |  0.014916538 |            f
   60 |    3150 |  0.00000E+00 |  0.00000E+00 |      25 |  0.009602555 |            f
   61 |    3200 |  0.00000E+00 |  0.00000E+00 |      27 |  0.016064971 |        nadir
   62 |    3250 |  0.00000E+00 |  0.00000E+00 |  